In [1]:
import numpy as np 
import pandas as pd

In [2]:
BASE_PATH = r"D:\DATA_ANALYST\FULL_STACK_FROJECT\Stock Market Prediction\stock_market_unclean_dataset"
df = pd.read_csv(BASE_PATH + r"\raw_data\bse_prices.csv")

In [3]:
df.head()

,date,market,company,open,high,low,close
0,2015-01-01,BSE,HDFC,1259.00,1310.79,1248.43,1295.84
1,2015-01-02,BSE,HDFC,1285.52,1299.75,1260.43,1276.72
2,2015-01-05,BSE,HDFC,1276.70,1292.93,1262.09,1270.12
3,2015-01-06,BSE,HDFC,1273.11,1292.21,1258.20,1261.92
4,2015-01-07,BSE,HDFC,1269.09,1298.18,1244.26,1274.37


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144600 entries, 0 to 144599
Data columns (total 7 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   date     144600 non-null  object 
 1   market   144600 non-null  object 
 2   company  144600 non-null  object 
 3   open     144600 non-null  float64
 4   high     144600 non-null  float64
 5   low      144600 non-null  float64
 6   close    144600 non-null  float64
dtypes: float64(4), object(3)
memory usage: 7.7+ MB


In [5]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df.dropna(subset=['date'], inplace=True)
df.sort_values(by=['company', 'date'], inplace=True)

In [6]:
df['company'] = df['company'].str.strip().str.upper()

In [7]:
df['close'] = df.groupby('company')['close'].ffill()

In [8]:
df['close'] = df.groupby('company')['close'].bfill()

In [9]:
df = df[
    (df['high'] >= df['low']) &
    (df['high'] >= df['open']) &
    (df['high'] >= df['close']) &
    (df['low']  <= df['open']) &
    (df['low']  <= df['close'])
]


In [10]:
price_cols = ['open', 'high', 'low', 'close']
df = df[(df[price_cols] > 0).all(axis=1)]

In [11]:
df.drop_duplicates(subset=['date', 'company'], inplace=True)

In [12]:
df = df[
    (df['open'] < df['open'].quantile(0.999)) &
    (df['close'] < df['close'].quantile(0.999))
]

In [13]:
df.reset_index(drop=True, inplace=True)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144450 entries, 0 to 144449
Data columns (total 7 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   date     144450 non-null  datetime64[ns]
 1   market   144450 non-null  object        
 2   company  144450 non-null  object        
 3   open     144450 non-null  float64       
 4   high     144450 non-null  float64       
 5   low      144450 non-null  float64       
 6   close    144450 non-null  float64       
dtypes: datetime64[ns](1), float64(4), object(2)
memory usage: 7.7+ MB


In [15]:
df.head()

,date,market,company,open,high,low,close
0,2015-01-01,BSE,ADANIPOWER,2170.60,2188.75,2101.05,2136.60
1,2015-01-02,BSE,ADANIPOWER,2096.16,2131.53,2036.74,2041.01
2,2015-01-05,BSE,ADANIPOWER,2130.46,2148.56,2116.96,2140.88
3,2015-01-06,BSE,ADANIPOWER,2183.04,2208.37,2152.16,2181.51
4,2015-01-07,BSE,ADANIPOWER,2222.31,2286.97,2179.16,2247.23


In [16]:
df = df.rename(columns={'date': 'trade_date'})
df = df[['company', 'trade_date', 'open', 'high', 'low', 'close']]

In [17]:
df.to_csv("base_price.csv", index=False)